In [1]:
import sys
from collections import defaultdict
from collections import Counter
#import ftfy
import re
import numpy as np


In [2]:
class Word:
    def __init__(self, word, upos, lemma=None, xpos=None, feats=None, misc=None):
        self.word = word
        self.norm = strong_normalize(word)
        self.lemma = lemma if lemma else "_"
        self.upos = upos
        self.xpos = xpos if xpos else "_"
        self.xupos = self.upos + "|" + self.xpos
        self.feats = feats if feats else "_"
        self.feats_set = "_" #parse_features(self.feats)
        self.misc = misc if misc else "_"
        #self.lang = lang if lang else "_"
    

class FakeNews:
    def __init__(self, id, source, label, cite, author, lang_code, title, content, date, references):
        self.id = id
        self.source = source
        self.label = label
        self.cite = cite
        self.author = author
        self.lang_code = lang_code
        self.title = title
        self.content = content
        self.date = date
        self.references = references
        
        self.title_tokenized = []
        self.content_tokenized = []
        
        
    def show(self):
        print("ID: ", self.id)
        print("Source: ", self.source)
        print("Label: ", self.label)
        print("Title: ", self.title)
        print("Content: ", self.content)
        
    def tokenization(self):
        tokenized = self.title.split(" ")
        self.title_tokenized = [Word(token, "Noun") for token in tokenized]
        tokenized = self.content.split(" ")
        self.content_tokenized = [Word(token, "Noun") for token in tokenized]
        #print(token_list)
        

In [3]:
def read_fakenews(file, limit=1000000):
    print("Read file: ", file)
    data_list = []
    
    with open(file=file, encoding="UTF-8") as file:
        idx = 0
        while True:
            idx+=1
            line = file.readline().split("\t")
            
            if line[0] is "":
                print("Finish to read: ", len(data_list), " sets")
                break
            else:
                data = FakeNews(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8],line[9])
                data.tokenization()
                #data.tagging()
                #data.parsing()
                data_list.append(data)
                if idx > limit:
                    break
    return data_list


def build_voca(data_list, cutoff=1):
    print("build vocaburaries")
    
    wordsCount = Counter()
    charsCount = Counter()
    uposCount = Counter()
    labelCount = Counter()
    
    for data in data_list:
        wordsCount.update([token.norm for token in data.title_tokenized])
        wordsCount.update([token.norm for token in data.content_tokenized])
        for token in data.title_tokenized:
            charsCount.update(token.word)
        for token in data.content_tokenized:
            charsCount.update(token.word)
        
        uposCount.update([token.upos for token in data.title_tokenized])
        uposCount.update([token.upos for token in data.content_tokenized])
        
        labelCount.update([data.label])
    
    wordsCount = Counter({w: i for w, i in wordsCount.items() if i >= cutoff})
    print("Vocab containing {} words".format(len(wordsCount)))
    print("Charset containing {} chars".format(len(charsCount)))
    print("UPOS containing {} tags".format(len(uposCount)), uposCount)
    print("LABEL containing {} tags".format(len(labelCount)), labelCount)
 
    full_dictionary = {
        "vocab": list(wordsCount.keys()),
        "wordfreq": wordsCount,
        "charset": list(charsCount.keys()),
        "charfreq": charsCount,
        "upos": list(uposCount.keys()),
        "uposfreq": uposCount.keys(),
        "label": list(labelCount.keys())
    }
    
    return full_dictionary

    
def strong_normalize(word):
    w = word.lower()
    w = re.sub(r".+@.+", "*EMAIL*", w)
    w = re.sub(r"@\w+", "*AT*", w)
    w = re.sub(r"(https?://|www\.).*", "*url*", w)
    w = re.sub(r"([^\d])\1{2,}", r"\1\1", w)
    w = re.sub(r"([^\d][^\d])\1{2,}", r"\1\1", w)
    w = re.sub(r"``", '"', w)
    w = re.sub(r"''", '"', w)
    w = re.sub(r"\d", "0", w)
    return w

In [4]:
#data_list = read_fakenews("./corpus/train.txt")

In [5]:
#full_dic = build_voca(data_list)
#data_list[0].title_tokenized[0].upos

In [6]:
# Pytorch Models
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class Encode_model(nn.Module):
    def __init__(self, 
                 num_word,
                 dim_word,# = 100,
                 num_extWord,
                 dim_extWord,# = 100,
                 num_pos,
                 dim_pos,# = 50,
                 num_label,
                 dropout_ratio,# = 0.33,
                 dim_lstm_hidden,# = 50,
                 num_lstm_layer,# = 50,
                 cuda_device# = 0
                ):
        super(Encode_model, self).__init__()
        
        self.dim_pos = dim_pos
        self.dim_lstm_hidden = dim_lstm_hidden
        self.num_lstm_layer = num_lstm_layer
        
        self.word_emb = nn.Embedding(num_embeddings=num_word + 3, embedding_dim=dim_word)
        self.pos_emb = nn.Embedding(num_embeddings=num_pos + 3, embedding_dim=dim_pos) if dim_pos > 0 else None
        
        self.input_size = dim_word + dim_pos
        self.enc_lstm = nn.LSTM(input_size=self.input_size, hidden_size=dim_lstm_hidden)
        self.enc_lstm_hidden = self.init_hidden()
        self.linear_classifier = nn.Linear(in_features=dim_lstm_hidden, out_features=num_label)
        
    def forward(self, word_seqs):
        
        word_seqs = torch.LongTensor(word_seqs)
        word_vecs = self.word_emb(word_seqs)
#        if torch.cuda.is_available() and self.cuda_device >= 0:
#            word_vecs.cuda()
        
        if self.dim_pos > 0:
            pos_vecs = self.pos_emb(pos_seq)
            input_vecs = torch.cat(word_vecs, pos_vecs)
        else:
            input_vecs = word_vecs
            
        lstm_out, lstm_hidden = self.enc_lstm(word_vecs.view(len(word_vecs), 1, -1), self.enc_lstm_hidden)
        #print(lstm_out)
        #print(lstm_hidden)
        lin_out = self.linear_classifier(lstm_hidden[0])
        #lin_out2 = self.linear_classifier(lstm_out.view(len(word_seqs), -1))

        return lin_out.squeeze()
    
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(self.num_lstm_layer, 1, self.dim_lstm_hidden),
                torch.zeros(self.num_lstm_layer, 1, self.dim_lstm_hidden))

In [7]:
class Fakefinder:
    def __init__(self, train_file, test_file, n_read, cuda_device=-1):
        
        self.train_file = train_file
        self.test_file = test_file
        self.train_data_list = read_fakenews(self.train_file, limit=n_read)
        self.test_data_list = read_fakenews(self.test_file, limit=n_read)
        self.full_dic = build_voca(self.train_data_list)
        self._set_vocab(self.full_dic)
        
        self.num_word = len(self._vocab)
        self.dim_word = 100
        self.num_extWord = len(self._vocab)
        self.dim_extWord = 100
        self.num_pos = len(self._upos)
        self.dim_pos = 0
        self.num_label = 3

        self.learning_rate = 0.01
        self.dropout_ratio = 0.33
        self.dim_lstm_hidden = 200
        self.num_lstm_layer = 1
        self.cuda_device = cuda_device
        
        self.enc_model = Encode_model(
                                    num_word = self.num_word,
                                    dim_word = self.dim_word,
                                    num_extWord = self.num_extWord,
                                    dim_extWord = self.dim_extWord,
                                    num_pos = self.num_pos ,
                                    dim_pos = self.dim_pos,
                                    num_label = self.num_label,
                                    dropout_ratio = self.dropout_ratio,
                                    dim_lstm_hidden = self.dim_lstm_hidden,
                                    num_lstm_layer = self.num_lstm_layer,
                                    cuda_device = self.cuda_device)
        
        self.optimizer = torch.optim.SGD(self.enc_model.parameters(), lr=self.learning_rate)
        self.criterion = nn.CrossEntropyLoss()
        
        if torch.cuda.is_available() and self.cuda_device >= 0:
            self.enc_model.cuda(self.cuda_device)
    
    def _set_vocab(self, vocab):
    
        self._fullvocab = vocab
        self._upos = {p: i for i, p in enumerate(vocab["upos"])}
        self._iupos = vocab["upos"]
        self._vocab = {w: i + 3 for i, w in enumerate(vocab["vocab"])}
        self._wordfreq = vocab["wordfreq"]
        self._charset = {c: i + 3 for i, c in enumerate(vocab["charset"])}
        self._charfreq = vocab["charfreq"]
        self._label = {w: i for i, w in enumerate(vocab["label"])}
        self._ilabel = vocab["label"]
        
        self._vocab['*pad*'] = 0
        self._charset['*pad*'] = 0
        
        self._vocab['*root*'] = 1
        self._charset['*root*'] = 1

    def train(self, batch_size=32):
        
        print("Running train sets")    
        self.enc_model.train()
        num_train = 0
        num_accurate = 0
        total_loss = 0
        
        for data in self.train_data_list:
            self.enc_model.zero_grad()
            num_train+=1
            
            words = [token.norm for token in data.content_tokenized]
            word_seqs = [self._vocab.get(token.norm, 0) for token in data.content_tokenized]
            label = [self._label.get(data.label)]
            #print(words[0])
            #print(word_seqs)
            #print(label_seq)
            
            logists = self.enc_model(word_seqs)
            #print(logists)
            loss = self.compute_loss(logists, label)
            accurate = self.compute_accuracy(logists, label) 
            num_accurate += accurate
            
            #print("accuracy", accuracy)
            #print("loss", loss.data[0])
            total_loss += loss.item()
            loss.backward()
            self.optimizer.step()
        print("Training loss: ", round(total_loss,2),  "(",num_train,"/",num_accurate.item(),")", "  accuracy: ", round((num_accurate.item()/num_train)*100,2) )

            
    def compute_loss(self, pred_logist, gold):
        
        #gold = gold[0]
        #gold = np.eye(3)[gold]
        gold = torch.LongTensor(gold)
        #print(gold)
        #print(pred_logist.squeeze())
        #print(F.softmax(pred_logist.squeeze()))
        pred = pred_logist.unsqueeze(dim=0)
        #print(pred)
        #loss = F.cross_entropy(pred_logist.squeeze(), F.softmax(gold))
        loss = self.criterion(pred, gold)
        return loss
    
    def compute_accuracy(self, pred_logist, gold):
        
        #value, predicted = torch.max(pred_logist.data, 1)
        #print(pred_logist.max())
        #print(pred_logist.max(0))
        pred = pred_logist.data.max(0)[1].cpu()
        #print("predicted " ,pred.data)
        #print("predicted2 " ,pred.data[0].cpu())
        #print("gold " ,gold[0])
        predicted = (pred == gold[0]) if gold[0] is not None else 0
        return predicted

    def test(self, test_file=None):
        print("Running test sets")
        self.enc_model.train()
        
        num_test = 0
        num_accurate = 0
        
        test_data_list = read_fakenews(test_file) if self.test_data_list is None else self.test_data_list
        #print(test_data_list[0].show())
        for data in test_data_list:
            num_test+=1
            
            words = [token.norm for token in data.content_tokenized]
            word_seqs = [self._vocab.get(token.norm, 0) for token in data.content_tokenized]
            label = [self._label.get(data.label)]
            #print(label)

            logists = self.enc_model(word_seqs)
            #print(logists)
            accurate = self.compute_accuracy(logists, label)
            #print(accurate)
            num_accurate += accurate

        print("Testing results: ", "(",num_test,"/",num_accurate.item(),")", "  accuracy: ", round((num_accurate.item()/num_test)*100,2) )

        
    def test_input(self, title, content):
        print("Running with an input text")
        self.enc_model.eval()
        
        num_test = 0
        num_accurate = 0
        #input_data = FakeNews(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8],line[9])
        input_data = FakeNews("","","","","","","",content,title,"")
        input_data.tokenization()
        
        test_data_list = [input_data]
        print(test_data_list[0].show())
        for data in test_data_list:
            num_test+=1
            
            words = [token.norm for token in data.content_tokenized]
            word_seqs = [self._vocab.get(token.norm, 0) for token in data.content_tokenized]

            logists = self.enc_model(word_seqs)
            print("logists", logists)
            softLog = F.softmax(logists)
            print("예측값:", softLog)
            #print(softLog.cpu()[0])
            #print(softLog[1].item())
            for idx, label in enumerate(self._ilabel):
                print(label,":", round(softLog[idx].item(),3))

     
    

In [8]:
fakefinder = Fakefinder("./corpus/train_ko.txt", "./corpus/dev.txt", n_read=40)



Read file:  ./corpus/train_ko.txt
Finish to read:  10  sets
Read file:  ./corpus/dev.txt
build vocaburaries
Vocab containing 1713 words
Charset containing 586 chars
UPOS containing 1 tags Counter({'Noun': 2386})
LABEL containing 2 tags Counter({'fakeNews': 6, 'trusted': 4})


In [10]:

while range(100):
    fakefinder.train()
    fakefinder.test()
    fakefinder.test_input("부동산시장 어디로?…'집코노미 콘서트'에서 해답찾으세요.",
                          "[ 안혜원 기자 ] 지난해 나온 ‘9·13 부동산 대책’ 이후 서울·수도권 부동산시장이 힘을 쓰지 못하다가 이달 들어 서울 강남권 일부 단지가 반등에 성공하고 있다. 바닥을 확인했다는 분석과 반짝 반등에 불과하다는 의견이 대립하고 있다. 봄 성수기를 맞아 집값이 본격적으로 반등할 것인가, 더 떨어질 것인가. 한국경제신문과 한경닷컴은 이에 대한 해답을 얻기 위해 다음달 26일 최고의 부동산 전문가들을 초청해 ‘제2회 한경 집코노미 부동산 콘서트’를 연다. 콘서트 주제는 ‘올해 부동산 시장 어디로?’다. 다양한 견해를 듣기 위해 올해 집값이 강세를 보일 것으로 예상한 전문가와 약세일 것으로 전망한 전문가, 그리고 공시가격 급등이 집값에 미칠 영향을 분석할 세무 전문가를 강사로 초빙했다. 빅데이터 전문가와 함께 저평가 지역을 가려내는 토크쇼도 마련했다. 이상우 유진투자증권 연구위원이 첫 번째로 연단에 오른다. 이 연구위원은 2017년과 2018년 집값 상승률을 정확하게 예측해 부동산 시장의 ‘족집게’로 불린다. 올해도 8% 상승을 예상한 그는 이번 강연에선 ‘올해 부동산 투자의 핵심 키워드’를 짚어준다. 이어 원종훈 KB국민은행 세무팀장이 증여 등을 활용한 보유세 절세 전략을 강연한다. 주택임대사업자 등록에 따른 구체적 실익도 살펴볼 계획이다. 채상욱 하나금융투자 연구위원은 ‘규제의 시대 부동산 어떻게 해야 하나’를 주제로 강연한다. 채 연구위원은 올해 서울 집값이 8% 하락할 것으로 예상한 바 있다. 그는 약세장에서 지역과 아파트 면적, 공시가격 등에 따라 집값이 어떻게 차별화되는지 분석해 대응법을 제공한다. 마지막 순서인 토크쇼에선 빅데이터 전문가인 조영광 대우건설 연구원과 최진석 기자가 빅데이터를 활용해 저평가된 지역을 찾는 방법과 현재 저평가된 지역을 소개한다. 조 과장은 “적정가치 대비 싼값에 팔리는 부동산을 찾아볼 것”이라고 말했다. 행사는 논현동 건설회관 2층 대회의실에서 오후 1시부터 6시까지 열린다. 참가비는 5만5000원이다.")

Running train sets
Training loss:  8.82 ( 10 / 10 )   accuracy:  100.0
Running test sets
Testing results:  ( 41 / 18 )   accuracy:  43.9
Running with an input text
ID:  
Source:  
Label:  
Title:  
Content:  [ 안혜원 기자 ] 지난해 나온 ‘9·13 부동산 대책’ 이후 서울·수도권 부동산시장이 힘을 쓰지 못하다가 이달 들어 서울 강남권 일부 단지가 반등에 성공하고 있다. 바닥을 확인했다는 분석과 반짝 반등에 불과하다는 의견이 대립하고 있다. 봄 성수기를 맞아 집값이 본격적으로 반등할 것인가, 더 떨어질 것인가. 한국경제신문과 한경닷컴은 이에 대한 해답을 얻기 위해 다음달 26일 최고의 부동산 전문가들을 초청해 ‘제2회 한경 집코노미 부동산 콘서트’를 연다. 콘서트 주제는 ‘올해 부동산 시장 어디로?’다. 다양한 견해를 듣기 위해 올해 집값이 강세를 보일 것으로 예상한 전문가와 약세일 것으로 전망한 전문가, 그리고 공시가격 급등이 집값에 미칠 영향을 분석할 세무 전문가를 강사로 초빙했다. 빅데이터 전문가와 함께 저평가 지역을 가려내는 토크쇼도 마련했다. 이상우 유진투자증권 연구위원이 첫 번째로 연단에 오른다. 이 연구위원은 2017년과 2018년 집값 상승률을 정확하게 예측해 부동산 시장의 ‘족집게’로 불린다. 올해도 8% 상승을 예상한 그는 이번 강연에선 ‘올해 부동산 투자의 핵심 키워드’를 짚어준다. 이어 원종훈 KB국민은행 세무팀장이 증여 등을 활용한 보유세 절세 전략을 강연한다. 주택임대사업자 등록에 따른 구체적 실익도 살펴볼 계획이다. 채상욱 하나금융투자 연구위원은 ‘규제의 시대 부동산 어떻게 해야 하나’를 주제로 강연한다. 채 연구위원은 올해 서울 집값이 8% 하락할 것으로 예상한 바 있다. 그는 약세장에서 지역과 아파트 면적, 공시가격 등에 따라 집값이 어떻게 차별화되는지 분석해 대응법을

/anaconda3/envs/NLPApps/lib/python3.7/site-packages/ipykernel_launcher.py:167: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Training loss:  8.41 ( 10 / 10 )   accuracy:  100.0
Running test sets
Testing results:  ( 41 / 18 )   accuracy:  43.9
Running with an input text
ID:  
Source:  
Label:  
Title:  
Content:  [ 안혜원 기자 ] 지난해 나온 ‘9·13 부동산 대책’ 이후 서울·수도권 부동산시장이 힘을 쓰지 못하다가 이달 들어 서울 강남권 일부 단지가 반등에 성공하고 있다. 바닥을 확인했다는 분석과 반짝 반등에 불과하다는 의견이 대립하고 있다. 봄 성수기를 맞아 집값이 본격적으로 반등할 것인가, 더 떨어질 것인가. 한국경제신문과 한경닷컴은 이에 대한 해답을 얻기 위해 다음달 26일 최고의 부동산 전문가들을 초청해 ‘제2회 한경 집코노미 부동산 콘서트’를 연다. 콘서트 주제는 ‘올해 부동산 시장 어디로?’다. 다양한 견해를 듣기 위해 올해 집값이 강세를 보일 것으로 예상한 전문가와 약세일 것으로 전망한 전문가, 그리고 공시가격 급등이 집값에 미칠 영향을 분석할 세무 전문가를 강사로 초빙했다. 빅데이터 전문가와 함께 저평가 지역을 가려내는 토크쇼도 마련했다. 이상우 유진투자증권 연구위원이 첫 번째로 연단에 오른다. 이 연구위원은 2017년과 2018년 집값 상승률을 정확하게 예측해 부동산 시장의 ‘족집게’로 불린다. 올해도 8% 상승을 예상한 그는 이번 강연에선 ‘올해 부동산 투자의 핵심 키워드’를 짚어준다. 이어 원종훈 KB국민은행 세무팀장이 증여 등을 활용한 보유세 절세 전략을 강연한다. 주택임대사업자 등록에 따른 구체적 실익도 살펴볼 계획이다. 채상욱 하나금융투자 연구위원은 ‘규제의 시대 부동산 어떻게 해야 하나’를 주제로 강연한다. 채 연구위원은 올해 서울 집값이 8% 하락할 것으로 예상한 바 있다. 그는 약세장에서 지역과 아파트 면적, 공시가격 등에 따라 집값이 어떻게 차별화되는지 분석해 대응법을 제공한다. 마지막 순서인 토크쇼에

KeyboardInterrupt: 